In [1]:
# Import required libraries

import pandas as pd
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB
import json

import time

# Record the start time
start_time = time.time()

In [2]:
# Inputs

no_of_customers = 7 # Total number of customers
no_of_depots = 2 # Total number of depots
total_vehicles = 4 # Total number of vehicles
trips_per_vehicle = 3 # Maximum number of trips per vehicle
D_V = {8:[0,1], 9:[2,3]} # Depot to vehicle mapping

# Vehicles
K = []
for i, j in D_V.items():
    K = K + j
    
time_window = [60,240] # Time window at which customers are served
H = 300 # Time horizon
s = 5 # service time at customer
service_at_depot = 5 # service time at depot

In [3]:
C = list(range(1, no_of_customers + 1))
D = list(range(len(C) + 1, len(C) + no_of_depots + 1, 1))
W = list(range(1, trips_per_vehicle))
Q = 25

In [4]:
# Teminologies

N = C + D # Union of customers and depots
M = 10000

# Data

In [5]:
# Pickup for customers

with open('p.json', 'r') as f:
    p = json.load(f)

print(p)

[5, 5, 4, 4, 3, 3, 4, 0, 0]


In [6]:
# Delivery for customers

with open('q.json', 'r') as f:
    q = json.load(f)
    
print(q)

[5, 8, 9, 7, 5, 8, 8, 0, 0]


In [7]:
# Retrieve the dictionary from JSON

with open('customers.json', 'r') as json_file:
    customers = json.load(json_file)
    
print(customers)
    
# Retrieve the dictionary from JSON

with open('depots.json', 'r') as json_file:
    depots = json.load(json_file)
    
print(depots)

{'Cust_1': [20, 20], 'Cust_2': [1, 13], 'Cust_3': [10, 7], 'Cust_4': [20, 23], 'Cust_5': [1, 20], 'Cust_6': [3, 14], 'Cust_7': [4, 21]}
{'Depot_1': [6, 10], 'Depot_2': [23, 0]}


In [8]:
# Retrieve the matrix from JSON
with open('d.json', 'r') as json_file:
    d = json.load(json_file)

# Convert the nested list back to a NumPy array
d = np.array(d)

print(d)

[[ 0 26 23  3 19 23 17 24 23]
 [26  0 15 29  7  3 11  8 35]
 [23 15  0 26 22 14 20  7 20]
 [ 3 29 26  0 22 26 18 27 26]
 [19  7 22 22  0  8  4 15 42]
 [23  3 14 26  8  0  8  7 34]
 [17 11 20 18  4  8  0 13 40]
 [24  8  7 27 15  7 13  0 27]
 [23 35 20 26 42 34 40 27  0]]


In [9]:
with open('ear.json', 'r') as f:
    ear = json.load(f)
    
with open('lat.json', 'r') as f:
    lat = json.load(f)
    
print(ear)
print(lat)

[110, 93, 124, 137, 85, 101, 164]
[240, 100, 225, 238, 164, 147, 221]


# Model

In [10]:
# Setting up the problem:

m = gp.Model("VRP")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-29


In [11]:
# Decision variables:

x = m.addVars(N, N, K, W, lb=0.0, vtype=GRB.BINARY, name = "x")
y = m.addVars(K, W, lb=0.0, vtype=GRB.BINARY, name = "y")

t = m.addVars(C, K, W, lb = 0.0, vtype=GRB.INTEGER, name = "Arrival time at customer")
ts = m.addVars(K, W, lb = 0.0, vtype=GRB.INTEGER, name = "Depot start time")
te = m.addVars(K, W, lb = 0.0, vtype=GRB.INTEGER, name = "Depot end time")
l = m.addVars(K, W, lb = 0.0, vtype=GRB.INTEGER, name = "Load at depot")
L = m.addVars(N, K, W, lb = 0.0, vtype=GRB.INTEGER, name = "Load at customer")

In [12]:
# Constraints

# 1) Each customer is served exactly once

for i in C:
    m.addConstr(gp.quicksum(x[(i,j,k,w)] for j in N for k in K for w in W) == 1)

In [13]:
# 2) Each vehicle can have only one depot

for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(i,d,k,w)] for i in C for d in D) == y[(k,w)])
        
for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(d,i,k,w)] for d in D for i in C) == y[(k,w)])

In [14]:
# 3) A vehcile cannot travel from depot to depot

for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(i,j,k,w)] for i in D for j in D) == 0)
        
# A vehicle cannot travel from i to i 

for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(i,i,k,w)] for i in C) == 0)

In [15]:
# 4) Total customers served by a vehicle must be less than the number of customers available

for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(i,j,k,w)] for i in N for j in N) <= ((len(C)+1) * y[(k,w)]))

In [16]:
# 5) Subsequent trips can be operated only if current trip is operated

for k in K:
    for w in W:
        if(w < (trips_per_vehicle - 1)):
            m.addConstr(y[(k,w+1)] <= y[(k,w)])

In [17]:
# 6) Flow conservation constraint

for j in C:
    for k in K:
        for w in W:
            m.addConstr(gp.quicksum(x[i,j,k,w] for i in N) == gp.quicksum(x[j,i,k,w] for i in N))

In [18]:
# 7) Load carried from depot should be equal to the demand of customers served by that vehicle

for k in K:
    for w in W:
        m.addConstr(gp.quicksum(x[(i,j,k,w)] * q[j-1] for i in N for j in N) == l[(k,w)])

In [19]:
# 8) Load carried by vehicle from depot should be less than vehicle capacity

for k in K:
    for w in W:
        m.addConstr(l[(k,w)] <= (Q * y[(k,w)]))

In [20]:
# 9) Depot start time constraint

for i in D:
    for k in K:
        for w in W:
            m.addConstr(ts[(k,w)] >= (s - M * (1 - y[(k,w)])))

In [21]:
# 10) Depot start time for consequent trips

for i in D:
    for k in K:
        for w in W:
            if w < (trips_per_vehicle - 1):
                m.addConstr(ts[(k,w+1)] >= (te[(k,w)] + s - M * (1 - y[(k,w+1)])))

In [22]:
# 11) Time at which you reach a customer should be less than latest time

for j in C:
    for k in K:
        for w in W:
            m.addConstr(gp.quicksum(x[(i, j, k, w)] * lat[j-1] for i in N) >= t[(j, k, w)])

In [23]:
# 12) Time at which you reach customer should be more than earliest time

for i in C:
    for j in C:
        for k in K:
            for w in W:
                m.addConstr(t[(j,k,w)] >= (t[(i,k,w)] + s + d[i-1][j-1] - M * (1 - x[i,j,k,w])))
                
for i in C:
    for j in C:
        for k in K:
            for w in W:
                m.addConstr(t[(j,k,w)] >= (ear[i-1] + s + d[i-1][j-1] - M * (1 - x[i,j,k,w])))

In [24]:
# 13) Time at which you reach customer should be more than start time at depot

for i in D:
    for j in C:
        for k in K:
            for w in W:
                m.addConstr(t[(j,k,w)] >= (ts[(k,w)] + d[i-1][j-1] - M * (1 - x[i,j,k,w])))

In [25]:
# 14) Time at which you reach the depot

for i in C:
    for j in D:
        for k in K:
            for w in W:
                m.addConstr(te[(k,w)] >= (t[i,k,w] + s + d[i-1][j-1] - M * (1 - x[i,j,k,w])))

for i in C:
    for j in D:
        for k in K:
            for w in W:
                m.addConstr(te[(k,w)] >= (ear[i-1] + s + d[i-1][j-1] - M * (1 - x[i,j,k,w])))

In [26]:
# 15) Time horizon

for k in K:
    for w in W:
        m.addConstr(te[(k,w)] <= H)
        

for k in K:
    for w in W:
        m.addConstr(ts[(k,w)] <= H)

In [27]:
# 16) Vehicle depot mapping

for i in D:
    for k in K:
        if k not in D_V[i]:
            for w in W:
                m.addConstr(gp.quicksum(x[(i,j,k,w)] for j in C) == 0)
                
for i in D:
    for k in K:
        if k not in D_V[i]:
            for w in W:
                m.addConstr(gp.quicksum(x[(j,i,k,w)] for j in C) == 0)

In [28]:
# 17) Arrival time

for j in C:
    for k in K:
        for w in W:
            m.addConstr(t[(j,k,w)] >= gp.quicksum(ear[j-1] * x[i,j,k,w] for i in N))

In [29]:
# 18) Load carried at any customer node should be less than the capacity of vehicle

for i in N:
    for k in K:
        for w in W:
            m.addConstr(L[(i,k,w)] <= Q)

In [30]:
# 19) Load carreid from any customer node should be equal to previous node load along with delivery and pickup at current node

for i in N:
    for j in C:
        for k in K:
            for w in W:
                m.addConstr(L[(j,k,w)] >= (L[(i,k,w)] + p[j-1] - q[j-1] - M * (1 - x[i,j,k,w])))

In [31]:
# 20) Load at depot constraint

for i in D:
    for k in K:
        for w in W:
            m.addConstr(L[(i,k,w)] >= l[(k,w)] - M * (1 - gp.quicksum(x[(i,j,k,w)] for j in C)))

In [32]:
# Objective function

m.setObjective(gp.quicksum(x[(i,j,k,w)] * d[i-1][j-1] for i in N for j in N for k in K for w in W), GRB.MINIMIZE)

In [33]:
 # + gp.quicksum(te[(k,w)] for k in K for w in W) + gp.quicksum(ts[(k,w)] for k in K for w in W) + gp.quicksum(L[(i,k,w)] for i in N for k in K for w in W)

In [34]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2003 rows, 808 columns and 8584 nonzeros
Model fingerprint: 0xa46d680b
Variable types: 0 continuous, 808 integer (656 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 1152 rows and 288 columns
Presolve time: 0.09s
Presolved: 851 rows, 520 columns, 4636 nonzeros
Variable types: 0 continuous, 520 integer (456 binary)
Found heuristic solution: objective 278.0000000

Root relaxation: objective 4.641618e+01, 173 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   4

In [35]:
obj = m.getObjective()
print(obj.getValue())

96.0


In [36]:
# Access solution results

for var in m.getVars():
    if var.X > 0:  # Check if the variable's value is greater than 0
        print(f"{var.VarName}: {var.X}")

x[1,8,1,2]: 1.0
x[2,6,1,1]: 1.0
x[3,8,0,1]: 1.0
x[4,1,1,2]: 1.0
x[5,7,1,2]: 1.0
x[6,8,1,1]: 1.0
x[7,4,1,2]: 1.0
x[8,2,1,1]: 1.0
x[8,3,0,1]: 1.0
x[8,5,1,2]: 1.0
y[0,1]: 1.0
y[1,1]: 1.0
y[1,2]: 1.0
Arrival time at customer[1,1,2]: 240.0
Arrival time at customer[2,1,1]: 93.0
Arrival time at customer[3,0,1]: 124.0
Arrival time at customer[4,1,2]: 232.0
Arrival time at customer[5,1,2]: 164.0
Arrival time at customer[6,1,1]: 101.0
Arrival time at customer[7,1,2]: 209.0
Depot start time[0,1]: 5.0
Depot start time[0,2]: 300.0
Depot start time[1,1]: 5.0
Depot start time[1,2]: 118.0
Depot start time[2,2]: 300.0
Depot start time[3,2]: 300.0
Depot end time[0,1]: 300.0
Depot end time[0,2]: 300.0
Depot end time[1,1]: 113.0
Depot end time[1,2]: 300.0
Depot end time[2,1]: 287.0
Depot end time[2,2]: 300.0
Depot end time[3,1]: 287.0
Depot end time[3,2]: 300.0
Load at depot[0,1]: 9.0
Load at depot[1,1]: 16.0
Load at depot[1,2]: 25.0
Load at customer[1,0,1]: 25.0
Load at customer[1,0,2]: 25.0
Load at cust

In [37]:
# Your program logic goes here

# Calculate elapsed time at a later point in your code
elapsed_time = time.time() - start_time

# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 2.47 seconds


In [38]:
result_dict = {'V01': [8, 3, 8], 'V11': [8, 2, 6, 8], 'V12': [8, 5, 7, 4, 1, 8]}

In [39]:
def calc_obj(result_dict):
    
    tot = 0
    
    for i, j in result_dict.items():
        m = 0
        n = 1
        while (n < len(j)):
            tot += d[j[m] - 1][j[n] - 1]
            m += 1
            n += 1
        
    return tot

calc_obj(result_dict)

96